In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [43]:
#get data in diffrent sheets.
data_1 = pd.read_excel('my_data.xlsx', sheet_name='Data1')
data_2 = pd.read_excel('my_data.xlsx', sheet_name='Data2')
data_3 = pd.read_excel('my_data.xlsx', sheet_name='Data3')
data_4 = pd.read_excel('my_data.xlsx', sheet_name='Data4')
data_5 = pd.read_excel('my_data.xlsx', sheet_name='Data5')
data_6 = pd.read_excel('my_data.xlsx', sheet_name='Data6')
weight_data = pd.read_excel('my_data.xlsx', sheet_name='Data weight')

In [44]:
#merge data in sheets.
data = pd.concat([data_1, data_2, data_3, data_4, data_5, data_6], ignore_index=True)
data

,Date,Courier Name,Region,Waybill No,To State
0,1403/06/21,حمید رضا دهقان لمراسكی,H3,10118721533267,DELIVERED
1,1403/06/21,حمید رضا دهقان لمراسكی,H3,10118721526007,DELIVERED
2,1403/06/21,حمید رضا دهقان لمراسكی,H3,10118721546774,DELIVERED
3,1403/06/21,حسین فرخی مهر,D4,10118721533776,DELIVERED
4,1403/06/21,حسین فرخی مهر,D4,10118721543054,DELIVERED
...,...,...,...,...,...
77582,1403/07/20,مصطفی فرهادی,NaN,10118721951846,DELIVERED
77583,1403/07/20,عرفان نوروزیان,NaN,10118721952029,DELIVERED
77584,1403/07/20,علیرضا باقرزاده خانقاه,NaN,10118721952155,NOT_DELIVERED
77585,1403/07/20,محسن عابدی نیا,NaN,10118721951958,DELIVERED


In [45]:
weight_data

,Waybill No,Weight
0,10118721533267,0.5
1,10118721526007,1.0
2,10118721546774,0.5
3,10118721533776,1.0
4,10118721543054,1.0
...,...,...
70231,10118721951846,1.0
70232,10118721952029,1.0
70233,10118721952155,NaN
70234,10118721951958,1.0


In [46]:
#merge data and weight by the 'Waybill No' column
merge_data = pd.merge(data, weight_data, on='Waybill No', how='outer')
merge_data

,Date,Courier Name,Region,Waybill No,To State,Weight
0,1403/06/21,حمید رضا دهقان لمراسكی,H3,10118721533267,DELIVERED,0.5
1,1403/06/21,حمید رضا دهقان لمراسكی,H3,10118721526007,DELIVERED,1.0
2,1403/06/21,حمید رضا دهقان لمراسكی,H3,10118721546774,DELIVERED,0.5
3,1403/06/21,حسین فرخی مهر,D4,10118721533776,DELIVERED,1.0
4,1403/06/21,حسین فرخی مهر,D4,10118721543054,DELIVERED,1.0
...,...,...,...,...,...,...
77582,1403/07/20,مصطفی فرهادی,NaN,10118721951846,DELIVERED,1.0
77583,1403/07/20,عرفان نوروزیان,NaN,10118721952029,DELIVERED,1.0
77584,1403/07/20,علیرضا باقرزاده خانقاه,NaN,10118721952155,NOT_DELIVERED,NaN
77585,1403/07/20,محسن عابدی نیا,NaN,10118721951958,DELIVERED,1.0


In [47]:
#analyze the number of nulls
merge_data = pd.DataFrame(merge_data)
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77587 entries, 0 to 77586
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          77587 non-null  object 
 1   Courier Name  77585 non-null  object 
 2   Region        68294 non-null  object 
 3   Waybill No    77587 non-null  int64  
 4   To State      77587 non-null  object 
 5   Weight        77581 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 3.6+ MB


In [48]:
merge_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Waybill No,77587.0,1.011872e+13,118598.061809,1.011872e+13,1.011872e+13,1.011872e+13,1.011872e+13,1.011872e+13
Weight,77581.0,3.237515e+00,11.498255,1.000000e-01,1.000000e+00,1.000000e+00,2.000000e+00,1.000000e+03


In [49]:
merge_data.describe(include=object).T

,count,unique,top,freq
Date,77587,31,1403/07/08,3715
Courier Name,77585,104,محمد جهانی آلام,3592
Region,68294,222,H4,5361
To State,77587,2,DELIVERED,67547


In [51]:
#'Courier Name' & 'Weight' have few nulls
merge_data = merge_data.dropna(subset=['Courier Name','Weight'], how= 'any')

In [52]:
#give code to status of delivery
merge_data['To State'] = merge_data['To State'].map({'DELIVERED':1, 'NOT_DELIVERED':0})
#merge_data

In [53]:
#dates are in persian calender, we need these functions in next steps
import jdatetime
def convert_date(shamsi_date):
    sh_y, sh_m, sh_d = map(int, shamsi_date.split('/'))
    gregorian = jdatetime.date(sh_y, sh_m, sh_d).togregorian()
    return gregorian
def jdate(shams):
    y, m, d = map(int, shams.split('/'))
    date = jdatetime.date(y, m , d)
    return date
merge_data['M_Date'] = merge_data['Date'].apply(convert_date)
merge_data['M_Date'] = pd.to_datetime(merge_data['M_Date'], errors='coerce')
merge_data['Date'] = merge_data['Date'].apply(jdate)
merge_data

,Date,Courier Name,Region,Waybill No,To State,Weight,M_Date
0,1403-06-21,حمید رضا دهقان لمراسكی,H3,10118721533267,1,0.5,2024-09-11
1,1403-06-21,حمید رضا دهقان لمراسكی,H3,10118721526007,1,1.0,2024-09-11
2,1403-06-21,حمید رضا دهقان لمراسكی,H3,10118721546774,1,0.5,2024-09-11
3,1403-06-21,حسین فرخی مهر,D4,10118721533776,1,1.0,2024-09-11
4,1403-06-21,حسین فرخی مهر,D4,10118721543054,1,1.0,2024-09-11
...,...,...,...,...,...,...,...
77581,1403-07-20,هومان پیش آهنگ,NaN,10118721951874,1,1.0,2024-10-11
77582,1403-07-20,مصطفی فرهادی,NaN,10118721951846,1,1.0,2024-10-11
77583,1403-07-20,عرفان نوروزیان,NaN,10118721952029,1,1.0,2024-10-11
77585,1403-07-20,محسن عابدی نیا,NaN,10118721951958,1,1.0,2024-10-11


In [54]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77579 entries, 0 to 77586
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          77579 non-null  object        
 1   Courier Name  77579 non-null  object        
 2   Region        68287 non-null  object        
 3   Waybill No    77579 non-null  int64         
 4   To State      77579 non-null  int64         
 5   Weight        77579 non-null  float64       
 6   M_Date        77579 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 4.7+ MB


In [55]:
#mean of every day(daily base)
daily_avg_per_driver = merge_data.groupby(by=['Courier Name','M_Date','Date']).agg(
    avg_weight_per_driver = ('Weight', 'mean'),
    avg_num_of_delivery = ('Waybill No', 'count')).reset_index()
daily_avg_per_driver

,Courier Name,M_Date,Date,avg_weight_per_driver,avg_num_of_delivery
0,آراد سعیدی,2024-09-11,1403-06-21,1.134286,56
1,آراد سعیدی,2024-09-14,1403-06-24,1.177800,25
2,آراد سعیدی,2024-09-15,1403-06-25,1.784459,37
3,آراد سعیدی,2024-09-16,1403-06-26,1.780000,50
4,آراد سعیدی,2024-09-17,1403-06-27,1.198113,53
...,...,...,...,...,...
2438,یوسف مرادپور,2024-10-07,1403-07-16,1.166364,11
2439,یوسف مرادپور,2024-10-08,1403-07-17,4.750000,4
2440,یوسف مرادپور,2024-10-09,1403-07-18,0.983000,27
2441,یوسف مرادپور,2024-10-10,1403-07-19,1.750000,8


In [56]:
daily_avg_per_driver.describe(include='all').T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
Courier Name,2443,104,ایمان ابراهیمی,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M_Date,2443,NaN,NaN,NaN,2024-09-26 03:18:38.460908800,2024-09-11 00:00:00,2024-09-18 00:00:00,2024-09-26 00:00:00,2024-10-03 00:00:00,2024-10-11 00:00:00,NaN
Date,2443,31,1403-06-27,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN
avg_weight_per_driver,2443.0,NaN,NaN,NaN,5.055189,0.4725,1.317558,2.133333,4.101748,321.5,12.202084
avg_num_of_delivery,2443.0,NaN,NaN,NaN,31.755628,1.0,17.0,28.0,42.0,356.0,24.560853


In [58]:
daily_avg_per_driver.describe().T

,count,mean,min,25%,50%,75%,max,std
M_Date,2443,2024-09-26 03:18:38.460908800,2024-09-11 00:00:00,2024-09-18 00:00:00,2024-09-26 00:00:00,2024-10-03 00:00:00,2024-10-11 00:00:00,NaN
avg_weight_per_driver,2443.0,5.055189,0.4725,1.317558,2.133333,4.101748,321.5,12.202084
avg_num_of_delivery,2443.0,31.755628,1.0,17.0,28.0,42.0,356.0,24.560853


In [62]:
delivery_report = merge_data.groupby(by='Courier Name').agg(
    delivered = ('To State', lambda x: (x==1).sum()),
    not_delivered = ('To State', lambda x: (x==0).sum())
).reset_index()
delivery_report

,Courier Name,delivered,not_delivered
0,آراد سعیدی,857,163
1,آزاد نوری,757,118
2,اسفندیار پرنون,901,14
3,اسلام پرور,494,89
4,اصغر خان محمدی,669,152
...,...,...,...
99,یاسر ناصربخت,1215,139
100,یاور حاجی حسن,518,57
101,یدالله نجفی,467,81
102,یوسف صبوحی راد,568,96


In [63]:
delivery_report.describe().T

,count,mean,std,min,25%,50%,75%,max
delivered,104.0,649.442308,425.047977,94.0,397.75,571.5,829.00,3591.0
not_delivered,104.0,96.509615,56.655662,0.0,52.00,91.0,131.25,255.0


In [64]:
#percentage of deliveries
delivery_report['delivered_percent'] = np.round(delivery_report['delivered']/(delivery_report['delivered']+delivery_report['not_delivered'])*100)
delivery_report

,Courier Name,delivered,not_delivered,delivered_percent
0,آراد سعیدی,857,163,84.0
1,آزاد نوری,757,118,87.0
2,اسفندیار پرنون,901,14,98.0
3,اسلام پرور,494,89,85.0
4,اصغر خان محمدی,669,152,81.0
...,...,...,...,...
99,یاسر ناصربخت,1215,139,90.0
100,یاور حاجی حسن,518,57,90.0
101,یدالله نجفی,467,81,85.0
102,یوسف صبوحی راد,568,96,86.0


In [65]:
#recognise the drivers who has delivery percent lower than 80 percent
k = delivery_report.shape[0]
low_rate_names = []
for i in range(0,k):
    if delivery_report.iloc[i,3]<80:
        name = delivery_report.iloc[i,0]
        delivery_rate = delivery_report.iloc[i, 3]
        low_rate_names.append((name,delivery_rate))
low_rate_names = pd.DataFrame(low_rate_names,columns=['Driver_Name','Delivery_Rate'])
low_rate_names = low_rate_names.sort_values(by='Delivery_Rate',ascending=True).reset_index(drop=True)
low_rate_names

,Driver_Name,Delivery_Rate
0,میلاد قراقونی,67.0
1,امیر باقری ملاحاجلو,71.0
2,جعفر عزیزی,71.0
3,مهدی حسین زاده,71.0
4,محمد سلیمی,72.0
5,امیرحسین داداشی,73.0
6,سهیل علیخانی,73.0
7,سید حسین بطهائی پور,73.0
8,اصغر موسی لو,74.0
9,امیرحسین کاظمی,74.0


In [66]:
#'Region' column has a lot of null, we cant delete these rows, we need them.
merge_data['Region'] = merge_data['Region'].fillna('unknown')

In [68]:
region_report = merge_data.groupby(by=['Region','Courier Name','M_Date','Date']).agg(
    number_of_boxes = ('Waybill No', 'count'),
    delivered_boxes = ('To State', lambda x: (x==1).sum()),
    not_delivered_boxes = ('To State', lambda x: (x==0).sum())
).reset_index()
region_report.drop(region_report[region_report['Region'] == 'مرخصی'].index, inplace=True)
region_report['delivered_percent'] = np.round(region_report['delivered_boxes']/(region_report['delivered_boxes']+region_report['not_delivered_boxes'])*100)
region_report

,Region,Courier Name,M_Date,Date,number_of_boxes,delivered_boxes,not_delivered_boxes,delivered_percent
0,D1,امین محمدی,2024-10-01,1403-07-10,28,28,0,100.0
1,D1,امین محمدی,2024-10-02,1403-07-11,55,54,1,98.0
2,D1,امین محمدی,2024-10-03,1403-07-12,17,17,0,100.0
3,D1,امین محمدی,2024-10-10,1403-07-19,32,32,0,100.0
4,D1,عرفان نوروزیان,2024-09-24,1403-07-03,27,25,2,93.0
...,...,...,...,...,...,...,...,...
2437,unknown,یوسف مرادپور,2024-09-13,1403-06-23,17,15,2,88.0
2438,unknown,یوسف مرادپور,2024-09-20,1403-06-30,26,11,15,42.0
2439,unknown,یوسف مرادپور,2024-09-21,1403-06-31,13,10,3,77.0
2440,unknown,یوسف مرادپور,2024-09-27,1403-07-06,21,16,5,76.0


In [95]:
#total number of boxes in every region
reg_report = region_report.groupby(by=['Region']).agg(
    boxes = ('number_of_boxes','sum')
)
reg_report

,boxes
Region,
D1,1499
D10,2092
D11,733
D11_N15,30
D12,1147
...,...
Nkaraj,998
Nkaraj_N6,8
karaj,2059


In [71]:
region_summary = merge_data.groupby(by='Region').agg(
    number_of_boxes = ('Waybill No', 'count'),
    delivered_boxes = ('To State', lambda x: (x==1).sum()),
    not_delivered_boxes = ('To State', lambda x: (x==0).sum())
).reset_index()
region_summary.drop(region_summary[region_summary['Region'] == 'مرخصی'].index, inplace=True)
region_summary['delivered_percent'] = np.round(region_summary['delivered_boxes']/(region_summary['delivered_boxes']+region_summary['not_delivered_boxes'])*100)
region_summary

,Region,number_of_boxes,delivered_boxes,not_delivered_boxes,delivered_percent
0,D1,1499,1360,139,91.0
1,D10,2092,1794,298,86.0
2,D11,733,594,139,81.0
3,D11_N15,30,27,3,90.0
4,D12,1147,1062,85,93.0
...,...,...,...,...,...
217,Nkaraj,998,916,82,92.0
218,Nkaraj_N6,8,8,0,100.0
219,karaj,2059,1951,108,95.0
220,karaj_N1,18,15,3,83.0


In [72]:
region_summary.describe().T

,count,mean,std,min,25%,50%,75%,max
number_of_boxes,222.0,349.423423,966.369785,2.0,17.0,31.0,83.75,9292.0
delivered_boxes,222.0,304.243243,839.686833,2.0,15.0,28.0,75.25,7712.0
not_delivered_boxes,222.0,45.180180,138.226792,0.0,1.0,3.0,12.75,1580.0
delivered_percent,222.0,89.301802,9.545978,56.0,84.0,91.0,97.00,100.0


In [105]:
#Assuming that the number of postal packages will grow by 0.2 per month, we wrote a function that, upon receiving a date, tells you how many drivers you need on that date.
from datetime import datetime, timedelta
def target_drivers(growth_rate):
    
    target_date = input('please enter target date(yyyy/mm/dd): ')
    target_date = jdate(target_date)
    today = jdatetime.date.today()
    k = (target_date.year - today.year)*12 + (target_date.month - today.month)
    #-------------------------------------------------------------------------------
    first_day_current_month = today.replace(day=1)
    end_last_month = first_day_current_month - timedelta(days=1)
    first_last_month = end_last_month.replace(day=1)

    filtered_df = merge_data[(merge_data['Date'] >= first_last_month) & (merge_data['Date'] <= end_last_month)]
    current_boxes = filtered_df['Waybill No'].count()
    drivers_no = filtered_df['Courier Name'].unique().shape[0]
    ship_per_driver = current_boxes / drivers_no
    #-------------------------------------------------------------------------------
    target_boxes = current_boxes * (1 + growth_rate) ** k
    target_drivers = np.round(target_boxes / ship_per_driver)
    return target_drivers,drivers_no

drivers_in_target = target_drivers(0.2)
print('Total number of drivers you need in target month is:',drivers_in_target[0])
new_drivers = drivers_in_target[0] - drivers_in_target[1]
print('New drivers you need in target month is:',new_drivers)

please enter target date(yyyy/mm/dd):  1403/12/1


Total number of drivers you need in target month is: 212.0
New drivers you need in target month is: 110.0
